In [1]:
import numpy as np
import math
import os
from scipy.misc import imread
# In order to import caffe, one may have to add caffe in the PYTHONPATH
import caffe
import matplotlib.pyplot as plt

%matplotlib inline

caffe.set_mode_gpu()
caffe.set_device(0)

In [6]:
from caffe.proto import caffe_pb2
from google.protobuf import text_format
import PIL.Image
import scipy.misc
import time
import os
import pandas as pd
from glob import glob

In [3]:
pwd

u'/home/shubham/work/fishery'

In [13]:
ls

blend.csv               clip_more_train_8.csv  submit2.csv
blend_more_train.csv    clip_more_train.csv    submit3.csv
clip2.csv               courses/               submit4.csv
clip3.csv               data/                  submit5.csv
clip.csv                model/                 submit_complete.csv
clip_more_train_2.csv   model2/                submit_complete_more_train.csv
clip_more_train_3.csv   model3/                submit.csv
clip_more_train_85.csv  model4/
clip_more_train_88.csv  model.ipynb


In [14]:
cd data/test/

/home/shubham/work/fishery/data/test


In [140]:
cd ../

/home/shubham/work/fishery/data


In [12]:
pwd

u'/home/shubham/work/fishery'

In [15]:
images_small = glob("*.jpg")

In [16]:
images_small

['img_04479.jpg',
 'img_03928.jpg',
 'img_02422.jpg',
 'img_05979.jpg',
 'img_07060.jpg',
 'img_02540.jpg',
 'img_05293.jpg',
 'img_02838.jpg',
 'img_01379.jpg',
 'img_05459.jpg',
 'img_05196.jpg',
 'img_05359.jpg',
 'img_06684.jpg',
 'img_03942.jpg',
 'img_06097.jpg',
 'img_05775.jpg',
 'img_03270.jpg',
 'img_00816.jpg',
 'img_03333.jpg',
 'img_00708.jpg',
 'img_04324.jpg',
 'img_03996.jpg',
 'img_01595.jpg',
 'img_00772.jpg',
 'img_05613.jpg',
 'img_01388.jpg',
 'img_05612.jpg',
 'img_01090.jpg',
 'img_03055.jpg',
 'img_03346.jpg',
 'img_00484.jpg',
 'img_02625.jpg',
 'img_03776.jpg',
 'img_01384.jpg',
 'img_04408.jpg',
 'img_05740.jpg',
 'img_04921.jpg',
 'img_06955.jpg',
 'img_06916.jpg',
 'img_00765.jpg',
 'img_02602.jpg',
 'img_01930.jpg',
 'img_05341.jpg',
 'img_02384.jpg',
 'img_04423.jpg',
 'img_00138.jpg',
 'img_06273.jpg',
 'img_05824.jpg',
 'img_07173.jpg',
 'img_06424.jpg',
 'img_01031.jpg',
 'img_01717.jpg',
 'img_02305.jpg',
 'img_01237.jpg',
 'img_02479.jpg',
 'img_0456

In [21]:
cd ..

/home/shubham/work/fishery/data


In [22]:
pwd

u'/home/shubham/work/fishery/data'

In [23]:
images_large = glob("test_stg2/*.jpg")

In [24]:
images = images_small + images_large

In [25]:
len(images)

13153

In [26]:
images[0]

'img_04479.jpg'

In [27]:
def get_net(caffemodel, deploy_file, use_gpu=True):
    """
    Returns an instance of caffe.Net
    Arguments:
    caffemodel -- path to a .caffemodel file
    deploy_file -- path to a .prototxt file
    Keyword arguments:
    use_gpu -- if True, use the GPU for inference
    """
    if use_gpu:
        caffe.set_mode_gpu()
        caffe.set_device(0)

    # load a new model
    return caffe.Net(deploy_file, caffemodel, caffe.TEST)

In [28]:
def get_transformer(deploy_file, mean_file=None):
    """
    Returns an instance of caffe.io.Transformer
    Arguments:
    deploy_file -- path to a .prototxt file
    Keyword arguments:
    mean_file -- path to a .binaryproto file (optional)
    """
    network = caffe_pb2.NetParameter()
    with open(deploy_file) as infile:
        text_format.Merge(infile.read(), network)

    if network.input_shape:
        dims = network.input_shape[0].dim
    else:
        dims = network.input_dim[:4]

    t = caffe.io.Transformer(inputs={'data': dims})
    t.set_transpose('data', (2, 0, 1))  # transpose to (channels, height, width)

    # color images
    if dims[1] == 3:
        # channel swap
        t.set_channel_swap('data', (2, 1, 0))

    if mean_file:
        # set mean pixel
        with open(mean_file, 'rb') as infile:
            blob = caffe_pb2.BlobProto()
            blob.MergeFromString(infile.read())
            if blob.HasField('shape'):
                blob_dims = blob.shape
                assert len(blob_dims) == 4, 'Shape should have 4 dimensions - shape is "%s"' % blob.shape
            elif blob.HasField('num') and blob.HasField('channels') and \
                    blob.HasField('height') and blob.HasField('width'):
                blob_dims = (blob.num, blob.channels, blob.height, blob.width)
            else:
                raise ValueError('blob does not provide shape or 4d dimensions')
            pixel = np.reshape(blob.data, blob_dims[1:]).mean(1).mean(1)
            t.set_mean('data', pixel)

    return t

In [29]:
def load_image(path, height, width, mode='RGB'):
    """
    Load an image from disk
    Returns an np.ndarray (channels x width x height)
    Arguments:
    path -- path to an image on disk
    width -- resize dimension
    height -- resize dimension
    Keyword arguments:
    mode -- the PIL mode that the image should be converted to
        (RGB for color or L for grayscale)
    """
    image = PIL.Image.open(path)
    image = image.convert(mode)
    image = np.array(image)
    # squash
    image = scipy.misc.imresize(image, (height, width), 'bilinear')
    return image

In [30]:
def forward_pass(images, net, transformer, batch_size=None):
    """
    Returns scores for each image as an np.ndarray (nImages x nClasses)
    Arguments:
    images -- a list of np.ndarrays
    net -- a caffe.Net
    transformer -- a caffe.io.Transformer
    Keyword arguments:
    batch_size -- how many images can be processed at once
        (a high value may result in out-of-memory errors)
    """
    if batch_size is None:
        batch_size = 1

    caffe_images = []
    for image in images:
        if image.ndim == 2:
            caffe_images.append(image[:, :, np.newaxis])
        else:
            caffe_images.append(image)

    dims = transformer.inputs['data'][1:]

    scores = None
    for chunk in [caffe_images[x:x + batch_size] for x in xrange(0, len(caffe_images), batch_size)]:
        new_shape = (len(chunk),) + tuple(dims)
        if net.blobs['data'].data.shape != new_shape:
            net.blobs['data'].reshape(*new_shape)
        for index, image in enumerate(chunk):
            image_data = transformer.preprocess('data', image)
            net.blobs['data'].data[index] = image_data
        start = time.time()
        output = net.forward()[net.outputs[-1]]
        end = time.time()
        if scores is None:
            scores = np.copy(output)
        else:
            scores = np.vstack((scores, output))
        print 'Processed %s/%s images in %f seconds ...' % (len(scores), len(caffe_images), (end - start))

    return scores

In [31]:
cd ..

/home/shubham/work/fishery


In [33]:
ls

blend.csv               clip_more_train_8.csv  submit2.csv
blend_more_train.csv    clip_more_train.csv    submit3.csv
clip2.csv               courses/               submit4.csv
clip3.csv               data/                  submit5.csv
clip.csv                model/                 submit_complete.csv
clip_more_train_2.csv   model2/                submit_complete_more_train.csv
clip_more_train_3.csv   model3/                submit.csv
clip_more_train_85.csv  model4/
clip_more_train_88.csv  model.ipynb


In [34]:
net = get_net('model4/snapshot_iter_4260.caffemodel', 'model4/deploy.prototxt')

In [35]:
transformer = get_transformer('model4/deploy.prototxt', 'model4/mean.binaryproto')

In [36]:
images[:2]

['img_04479.jpg', 'img_03928.jpg']

In [162]:
images

['img_04479.jpg',
 'img_03928.jpg',
 'img_02422.jpg',
 'img_05979.jpg',
 'img_07060.jpg',
 'img_02540.jpg',
 'img_05293.jpg',
 'img_02838.jpg',
 'img_01379.jpg',
 'img_05459.jpg',
 'img_05196.jpg',
 'img_05359.jpg',
 'img_06684.jpg',
 'img_03942.jpg',
 'img_06097.jpg',
 'img_05775.jpg',
 'img_03270.jpg',
 'img_00816.jpg',
 'img_03333.jpg',
 'img_00708.jpg',
 'img_04324.jpg',
 'img_03996.jpg',
 'img_01595.jpg',
 'img_00772.jpg',
 'img_05613.jpg',
 'img_01388.jpg',
 'img_05612.jpg',
 'img_01090.jpg',
 'img_03055.jpg',
 'img_03346.jpg',
 'img_00484.jpg',
 'img_02625.jpg',
 'img_03776.jpg',
 'img_01384.jpg',
 'img_04408.jpg',
 'img_05740.jpg',
 'img_04921.jpg',
 'img_06955.jpg',
 'img_06916.jpg',
 'img_00765.jpg',
 'img_02602.jpg',
 'img_01930.jpg',
 'img_05341.jpg',
 'img_02384.jpg',
 'img_04423.jpg',
 'img_00138.jpg',
 'img_06273.jpg',
 'img_05824.jpg',
 'img_07173.jpg',
 'img_06424.jpg',
 'img_01031.jpg',
 'img_01717.jpg',
 'img_02305.jpg',
 'img_01237.jpg',
 'img_02479.jpg',
 'img_0456

In [164]:
len(images)

13153

In [37]:
IMAGE_PATH = '/home/shubham/work/fishery/data/'

In [43]:
caffe_images = []
for i in range(len(images)):
    if i%1000==0:
        start = time.time()
    if i<1000:
        caffe_images.append(load_image(IMAGE_PATH + "test/" + images[i], height=1280, width=640))
    else:
        caffe_images.append(load_image(IMAGE_PATH+images[i], height=1280, width=640))
    if i%1000==0:
        end = time.time()
        print 'Processed %s images in %f seconds ...' % (len(caffe_images), (end - start))

Processed 1 images in 0.028458 seconds ...
Processed 1001 images in 0.023550 seconds ...
Processed 2001 images in 0.023088 seconds ...
Processed 3001 images in 0.024224 seconds ...
Processed 4001 images in 0.038388 seconds ...
Processed 5001 images in 0.059003 seconds ...
Processed 6001 images in 0.122906 seconds ...
Processed 7001 images in 0.023992 seconds ...
Processed 8001 images in 0.035810 seconds ...
Processed 9001 images in 0.027485 seconds ...
Processed 10001 images in 0.028181 seconds ...
Processed 11001 images in 0.023843 seconds ...
Processed 12001 images in 0.032410 seconds ...
Processed 13001 images in 0.023926 seconds ...


In [44]:
caffe_images[:5]

[array([[[253, 254, 252],
         [251, 252, 253],
         [253, 253, 252],
         ..., 
         [  9,  67,  53],
         [  6,  63,  47],
         [  4,  63,  45]],
 
        [[253, 254, 253],
         [252, 253, 254],
         [253, 253, 252],
         ..., 
         [  9,  67,  53],
         [  6,  63,  47],
         [  4,  63,  45]],
 
        [[254, 255, 255],
         [254, 255, 255],
         [254, 254, 253],
         ..., 
         [  9,  66,  54],
         [  6,  63,  47],
         [  4,  62,  45]],
 
        ..., 
        [[ 34, 119,  77],
         [ 36, 121,  79],
         [ 37, 124,  80],
         ..., 
         [  0,  53,  36],
         [  0,  55,  34],
         [  1,  56,  35]],
 
        [[ 35, 120,  78],
         [ 36, 122,  80],
         [ 37, 124,  81],
         ..., 
         [  0,  53,  35],
         [  1,  55,  35],
         [  1,  56,  35]],
 
        [[ 35, 120,  78],
         [ 36, 122,  80],
         [ 37, 124,  81],
         ..., 
         [  0,  53,  35

In [45]:
scores = forward_pass(caffe_images, net, transformer, batch_size=128)

Processed 128/13153 images in 0.230879 seconds ...
Processed 256/13153 images in 0.190679 seconds ...
Processed 384/13153 images in 0.187401 seconds ...
Processed 512/13153 images in 0.188025 seconds ...
Processed 640/13153 images in 0.187475 seconds ...
Processed 768/13153 images in 0.199784 seconds ...
Processed 896/13153 images in 0.190176 seconds ...
Processed 1024/13153 images in 0.186822 seconds ...
Processed 1152/13153 images in 0.187779 seconds ...
Processed 1280/13153 images in 0.187184 seconds ...
Processed 1408/13153 images in 0.185791 seconds ...
Processed 1536/13153 images in 0.187684 seconds ...
Processed 1664/13153 images in 0.199536 seconds ...
Processed 1792/13153 images in 0.190183 seconds ...
Processed 1920/13153 images in 0.186686 seconds ...
Processed 2048/13153 images in 0.186751 seconds ...
Processed 2176/13153 images in 0.187857 seconds ...
Processed 2304/13153 images in 0.188053 seconds ...
Processed 2432/13153 images in 0.188772 seconds ...
Processed 2560/1315

In [46]:
scores.shape

(13153, 8)

In [47]:
scores[1]

array([ 0.91994017,  0.03020824,  0.00215994,  0.00184327,  0.00726561,
        0.00349648,  0.01843839,  0.01664793], dtype=float32)

In [42]:
scores[1]

array([ 0.68152612,  0.16054676,  0.00762545,  0.01896442,  0.03966559,
        0.02120898,  0.03104278,  0.03941992], dtype=float32)

In [48]:
submit = pd.DataFrame()

In [49]:
submit['image'] = images

In [50]:
images

['img_04479.jpg',
 'img_03928.jpg',
 'img_02422.jpg',
 'img_05979.jpg',
 'img_07060.jpg',
 'img_02540.jpg',
 'img_05293.jpg',
 'img_02838.jpg',
 'img_01379.jpg',
 'img_05459.jpg',
 'img_05196.jpg',
 'img_05359.jpg',
 'img_06684.jpg',
 'img_03942.jpg',
 'img_06097.jpg',
 'img_05775.jpg',
 'img_03270.jpg',
 'img_00816.jpg',
 'img_03333.jpg',
 'img_00708.jpg',
 'img_04324.jpg',
 'img_03996.jpg',
 'img_01595.jpg',
 'img_00772.jpg',
 'img_05613.jpg',
 'img_01388.jpg',
 'img_05612.jpg',
 'img_01090.jpg',
 'img_03055.jpg',
 'img_03346.jpg',
 'img_00484.jpg',
 'img_02625.jpg',
 'img_03776.jpg',
 'img_01384.jpg',
 'img_04408.jpg',
 'img_05740.jpg',
 'img_04921.jpg',
 'img_06955.jpg',
 'img_06916.jpg',
 'img_00765.jpg',
 'img_02602.jpg',
 'img_01930.jpg',
 'img_05341.jpg',
 'img_02384.jpg',
 'img_04423.jpg',
 'img_00138.jpg',
 'img_06273.jpg',
 'img_05824.jpg',
 'img_07173.jpg',
 'img_06424.jpg',
 'img_01031.jpg',
 'img_01717.jpg',
 'img_02305.jpg',
 'img_01237.jpg',
 'img_02479.jpg',
 'img_0456

In [201]:
submit.tail()

,image
13148,test_stg2/image_09672.jpg
13149,test_stg2/image_11612.jpg
13150,test_stg2/image_09074.jpg
13151,test_stg2/image_09030.jpg
13152,test_stg2/image_07348.jpg


In [51]:
scores[0].shape

(8,)

In [134]:
scores.shape

(13153, 8)

In [52]:
len(scores[:,7])

13153

In [53]:
columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [54]:
for i in range(len(columns)):
    submit[columns[i]] = scores[:,i]    

In [55]:
submit.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_04479.jpg,0.673837,0.096628,0.019906,0.010035,0.118771,0.066665,0.007471,0.006687
1,img_03928.jpg,0.919940,0.030208,0.002160,0.001843,0.007266,0.003496,0.018438,0.016648
2,img_02422.jpg,0.167242,0.739007,0.004578,0.006975,0.043073,0.015239,0.003819,0.020067
3,img_05979.jpg,0.021849,0.005476,0.005891,0.000067,0.960546,0.002751,0.000160,0.003260
4,img_07060.jpg,0.103924,0.049365,0.029048,0.006444,0.021466,0.563449,0.003142,0.223161


In [56]:
final = submit.sort(columns=['image'])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [57]:
final.tail()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
12146,test_stg2/image_12149.jpg,0.613087,0.098662,0.041531,0.008934,0.165994,0.018256,0.009091,0.044445
12515,test_stg2/image_12150.jpg,0.334859,0.179572,0.023407,0.009258,0.431499,0.006580,0.006682,0.008144
1019,test_stg2/image_12151.jpg,0.113792,0.382680,0.105079,0.031582,0.229878,0.008637,0.008116,0.120237
1509,test_stg2/image_12152.jpg,0.074562,0.190014,0.026483,0.033475,0.489103,0.039662,0.033023,0.113678
7847,test_stg2/image_12153.jpg,0.297988,0.291956,0.050605,0.028256,0.165412,0.006949,0.043960,0.114874


In [58]:
pwd

u'/home/shubham/work/fishery'

In [43]:
cd work/fishery/

[Errno 2] No such file or directory: 'work/fishery/'
/home/shubham/work/fishery


In [182]:
pwd

u'/home/shubham/work/fishery'

In [59]:
final2 = final.round(decimals=8)

In [60]:
final2.to_csv('submit_complete_more_train_full_image.csv', index=False)

In [66]:
pwd

u'/home/shubham/work/fishery'

In [61]:
clip = 0.9
classes = 8

def clip_csv(csv_file, clip, classes):
    # Read the submission file
    df = pd.read_csv(csv_file, index_col=0)

    # Clip the values
    df = df.clip(lower=(1.0 - clip)/float(classes - 1), upper=clip)
    
    # Normalize the values to 1
    df = df.div(df.sum(axis=1), axis=0)

    # Save the new clipped values
    df.to_csv('clip_more_train_full_image_9.csv')
    print(df.head(10))

In [62]:
clip_csv('submit_complete_more_train_full_image.csv', clip, classes)

                    ALB       BET       DOL       LAG       NoF     OTHER  \
image                                                                       
img_00005.jpg  0.014286  0.014286  0.014286  0.014286  0.900000  0.014286   
img_00007.jpg  0.899776  0.014282  0.014282  0.014531  0.014282  0.014282   
img_00009.jpg  0.877174  0.032857  0.013923  0.013923  0.013923  0.020352   
img_00018.jpg  0.900000  0.014286  0.014286  0.014286  0.014286  0.014286   
img_00027.jpg  0.454946  0.200185  0.013998  0.013998  0.066041  0.177248   
img_00030.jpg  0.672562  0.013767  0.013767  0.013767  0.224577  0.013767   
img_00040.jpg  0.427428  0.062173  0.013990  0.013990  0.024956  0.024670   
img_00046.jpg  0.900000  0.014286  0.014286  0.014286  0.014286  0.014286   
img_00053.jpg  0.900000  0.014286  0.014286  0.014286  0.014286  0.014286   
img_00071.jpg  0.761050  0.013703  0.013703  0.156730  0.013703  0.013703   

                  SHARK       YFT  
image                              
img

In [218]:
def blend_csv(csv_paths):
    if len(csv_paths) < 2:
        print("Blending takes two or more csv files!")
        return
    
    # Read the first file
    df_blend = pd.read_csv(csv_paths[0], index_col=0)
    
    # Loop over all files and add them
    for csv_file in csv_paths[1:]:
        df = pd.read_csv(csv_file, index_col=0)
        df_blend = df_blend.add(df)
        
    # Divide by the number of files
    df_blend = df_blend.div(len(csv_paths))

    # Save the blend file
    df_blend.to_csv('blend_more_train.csv')
    print(df_blend.head(10))

In [192]:
pwd

u'/home/shubham/work/fishery'

In [222]:
ls

blend.csv               clip_more_train.csv  submit2.csv
blend_more_train.csv    courses/             submit3.csv
clip2.csv               data/                submit4.csv
clip3.csv               model/               submit5.csv
clip.csv                model2/              submit_complete.csv
clip_more_train_2.csv   model3/              submit_complete_more_train.csv
clip_more_train_3.csv   model4/              submit.csv
clip_more_train_88.csv  model.ipynb


In [219]:
# Obviously replace this with two or more of your files
blend_csv(['/home/shubham/work/fishery/clip_more_train.csv', '/home/shubham/work/fishery/clip_more_train_2.csv', 
          '/home/shubham/work/fishery/clip_more_train_3.csv'])

                    ALB       BET       DOL       LAG       NoF     OTHER  \
image                                                                       
img_00005.jpg  0.010476  0.010476  0.010476  0.010476  0.926667  0.010476   
img_00007.jpg  0.919432  0.010411  0.010411  0.013570  0.010411  0.014941   
img_00009.jpg  0.818711  0.043061  0.017188  0.032041  0.019724  0.048581   
img_00018.jpg  0.922740  0.010441  0.010441  0.010441  0.010441  0.014614   
img_00027.jpg  0.213409  0.345519  0.013253  0.021519  0.123709  0.192015   
img_00030.jpg  0.513632  0.010447  0.010444  0.010222  0.389094  0.010222   
img_00040.jpg  0.237141  0.258403  0.014180  0.018431  0.066656  0.081343   
img_00046.jpg  0.925655  0.010468  0.010468  0.010468  0.011534  0.010468   
img_00053.jpg  0.926667  0.010476  0.010476  0.010476  0.010476  0.010476   
img_00071.jpg  0.747037  0.012467  0.010293  0.181401  0.010293  0.014483   

                  SHARK       YFT  
image                              
img

In [87]:
ls

blend.csv  clip.csv  fishery_model.ipynb  model3/      submit4.csv
clip2.csv  courses/  model/               submit2.csv  submit5.csv
clip3.csv  data/     model2/              submit3.csv  submit.csv


In [88]:
! head -n 5 blend.csv

image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
img_00005.jpg,0.0142857142857,0.0142857142857,0.0142857142857,0.0142857142857,0.9,0.0142857142857,0.0142857142857,0.0142857142857
img_00007.jpg,0.899247069361,0.0142800531531,0.0142800531531,0.0142800531531,0.0142800531531,0.0150726117207,0.0142800531531,0.0142800531531
img_00009.jpg,0.887845054102,0.0144998709044,0.014132462742,0.014132462742,0.014132462742,0.0269927612834,0.014132462742,0.014132462742
img_00018.jpg,0.848703691227,0.0145361291034,0.0137001279697,0.0137001279697,0.0137001279697,0.0499463323129,0.0137001279697,0.0320133354774
